In [108]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import butter, sosfiltfilt, sosfreqz  
from scipy.io import loadmat
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_curve, auc

from sklearn.decomposition import FastICA

In [109]:
data_dir = 'data'
bc4_2a_raw_name = 'bci_competition_4_2a_raw'
bc4_2a_processed_name = 'bci_competition_4_2a_processed'

bc4_2a_raw_dir = os.path.join(data_dir, bc4_2a_raw_name)
bc4_2a_processed_dir = os.path.join(data_dir, bc4_2a_processed_name)

processed_dataset_path = os.path.join(bc4_2a_processed_dir, 'A01T.npz')
processed_dataset_npz = np.load(processed_dataset_path)
print(processed_dataset_npz.files)
print(processed_dataset_npz['processed_motor_imagery_data'].shape)

['processed_motor_imagery_data', 'descriptions', 'onsets']
(273, 22, 750)


In [110]:
for i in range(1, 10):

    subject_name = 'A0' + str(i) + 'T'
    subject_path = os.path.join(bc4_2a_raw_dir, subject_name + '.gdf')
    subject_mne = mne.io.read_raw_gdf(subject_path)
    raw_data = subject_mne.get_data()[:22]
    X = raw_data.T  # Observations (mixed signal)
    ica = FastICA(n_components=22)
    S_ = ica.fit_transform(X)  # Recovered signals
    A_ = ica.mixing_  # Get estimated mixing matrix
    
    
    
    annotations = subject_mne.annotations
    annotations_list = list(annotations)
    
    ## Convert annotation dictionary into lists
    descriptions = []
    onsets = []

    for i in range(len(annotations_list) - 1):
        
        description_cur = annotations_list[i]['description']
        description_next = annotations_list[i + 1]['description']
        
        ## Check if the current trial is valid and has a known cue
        if description_cur == '768' and description_next in ['769', '770', '771', '772']:    
            onsets.append(annotations_list[i]['onset'])
            descriptions.append(int(description_next))
    
    descriptions = np.array(descriptions)
    onsets = np.array(onsets)
    
    ## Process by onsets to create epochs
    num_valid_trials = onsets.shape[0]

    ## Initialize processed data array
    processed_motor_imagery_data = np.zeros((num_valid_trials,
                                             num_channels,
                                             motor_imagery_trial_duration * sampling_f))
    break



Extracting EDF parameters from /Users/oliviazhao/Documents/GitHub/COGS-189-Final-Project/data/bci_competition_4_2a_raw/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


/Users/oliviazhao/opt/anaconda3/lib/python3.8/site-packages/mne/io/edf/edf.py:1131: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/Users/oliviazhao/opt/anaconda3/lib/python3.8/contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


In [111]:
raw_data.shape

(22, 672528)

In [67]:
S_.shape

(672528, 22)

In [68]:
A_.shape

(22, 22)

In [ ]:
S_ = S_.T

In [ ]:
plt.plot(S_[1])

In [ ]:
plt.plot(S_[0])


In [ ]:
plt.plot(S_[3])

In [ ]:
raw_data.shape



In [ ]:
raw_data.shape
ica = FastICA(n_components=3)
S_ = ica.fit_transform(raw_data.T)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

In [ ]:
# dataset = processed_dataset_npz['processed_motor_imagery_data']
# new_dataset = []
# for i in range(25):
#     new_dataset.append(list(dataset[i].flatten()))
# new_dataset = np.array(new_dataset)
    

In [ ]:
new_dataset.shape

In [ ]:
labels = processed_dataset_npz["descriptions"]


In [ ]:
X.shape

In [ ]:

ica = FastICA(n_components=3)
S_ = ica.fit_transform(X)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

# We can `prove` that the ICA model applies by reverting the unmixing.
assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)


In [40]:
S_.shape

(25, 22)

In [4]:
import numpy as np
np.random.seed(0)
from scipy import signal
from scipy.io import wavfile
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [70]:
np.random.seed(23)

# Number of samples
ns = np.linspace(0, 200, 1000)

S = np.array([np.sin(ns * 1),
              signal.sawtooth(ns * 1.9),
              np.random.random(len(ns))]).T

# Mixing matrix
A = np.array([[0.5, 1, 0.2],
              [1, 0.5, 0.4],
              [0.5, 0.8, 1]])

# Mixed signal matrix
X = S.dot(A).T

In [82]:
S.shape
A.shape
X.shape
ns.shape

(1000,)

In [5]:
def g(x):
    return np.tanh(x)

def g_der(x):
    return 1 - g(x) * g(x)

In [ ]:
S = ica(X, iterations=10)
# actual = mix_sources([s1,s2,s3])
# plot_mixture_sources_predictions(X, [s1, s2, s3], S)

In [75]:
subject_name = 'A01T.gdf'
subject_path = os.path.join(bc4_2a_raw_dir, subject_name)
subject_mne = mne.io.read_raw_gdf(subject_path)
raw_data = subject_mne.get_data()[:22]
X = raw_data  # Observations (mixed signal)

Extracting EDF parameters from /Users/oliviazhao/Documents/GitHub/COGS-189-Final-Project/data/bci_competition_4_2a_raw/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


/Users/oliviazhao/opt/anaconda3/lib/python3.8/site-packages/mne/io/edf/edf.py:1131: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
/Users/oliviazhao/opt/anaconda3/lib/python3.8/contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


In [98]:
# https://github.com/akcarsten/Independent_Component_Analysis

In [60]:
def whiten(x):
    
    # center
    mean = np.mean(x, axis=1, keepdims=True)
    centered =  x - mean
    
    # covariance matrix
    coVarM = np.cov(centered)

    # Single value decomposition
    U, S, V = np.linalg.svd(coVarM)

    # Calculate diagonal matrix of eigenvalues
    d = np.diag(1.0 / np.sqrt(S))

    # Calculate whitening matrix
    whiteM = np.dot(U, np.dot(d, U.T))

    # Project onto whitening matrix
    Xw = np.dot(whiteM, X)

    return Xw, whiteM

In [61]:
def fastIca(signals,  alpha = 1, thresh=1e-8, iterations=1000):
    m, n = signals.shape

    # Initialize random weights
    W = np.random.rand(m, m)

    for c in range(m):
            w = W[c, :].copy().reshape(m, 1)
            w = w / np.sqrt((w ** 2).sum())

            i = 0
            lim = 100
            while ((lim > thresh) & (i < iterations)):

                # Dot product of weight and signal
                ws = np.dot(w.T, signals)

                # Pass w*s into contrast function g
                wg = np.tanh(ws * alpha).T

                # Pass w*s into g prime
                wg_ = (1 - np.square(np.tanh(ws))) * alpha

                # Update weights
                wNew = (signals * wg.T).mean(axis=1) - wg_.mean() * w.squeeze()

                # Decorrelate weights              
                wNew = wNew - np.dot(np.dot(wNew, W[:c].T), W[:c])
                wNew = wNew / np.sqrt((wNew ** 2).sum())

                # Calculate limit condition
                lim = np.abs(np.abs((wNew * w).sum()) - 1)

                # Update weights
                w = wNew

                # Update counter
                i += 1

            W[c, :] = w.T
    return W

In [99]:
# Whiten mixed signals
Xw, whiteM = whiten(X)

In [100]:
W = fastIca(Xw)

In [107]:
# ica
W

array([[-1.58610954e-01, -2.18285175e-01, -1.94106923e-01,
        -2.01600746e-01, -2.04119977e-01, -1.99300498e-01,
        -2.94946367e-01, -2.14938682e-01, -1.96443511e-01,
        -1.88161077e-01, -1.86349348e-01, -1.99695488e-01,
        -2.64257821e-01, -2.13471291e-01, -1.99442104e-01,
        -2.07816336e-01, -1.98509895e-01, -2.03770085e-01,
        -2.22256566e-01, -1.87903829e-01, -2.31558638e-01,
        -2.60914736e-01],
       [ 3.71162373e-02,  4.57548013e-03,  1.95576011e-01,
        -2.11404383e-02,  2.74740489e-03,  1.92155709e-02,
         2.75316729e-02, -2.24703374e-01, -2.89054186e-01,
         3.33434314e-01, -1.14537202e-01, -3.90159248e-02,
        -2.19936410e-01,  1.21314151e-01, -6.37345885e-02,
        -7.09711496e-02, -1.43102403e-01,  6.90463305e-01,
        -4.30640104e-02, -4.98619375e-02, -2.91650164e-01,
         2.00249420e-01],
       [-1.06512928e-01, -2.87867548e-01,  6.97469982e-03,
        -2.46357401e-02,  2.83260799e-02,  2.63808794e-01,
    

In [106]:
# sklearn fastica
A_

array([[-5.94964589e-04, -7.76375797e-04, -6.62592581e-04,
         3.68559425e-04, -1.00792766e-03,  4.56328921e-02,
         9.41191565e-04,  1.44524036e-03, -2.24895393e-03,
        -4.83238813e-04, -5.27388695e-04, -8.67171562e-03,
         1.98453183e-04, -6.61293881e-04, -4.44253284e-04,
         3.19971186e-03,  9.11428100e-04, -2.45229454e-03,
        -2.61552973e-03, -1.97191611e-03,  1.71020347e-03,
        -1.73397930e-04],
       [-7.23212815e-04, -1.90818696e-04, -1.03716598e-03,
        -2.14922867e-04, -1.34819277e-03,  4.56634222e-02,
         6.86095413e-04,  1.23282939e-03, -1.38640093e-03,
         1.63384626e-03, -5.39582671e-04, -5.24679011e-03,
         2.09604615e-04,  8.64035283e-04, -5.39073048e-04,
         3.12206467e-03,  1.30831297e-03, -3.00385779e-03,
        -1.19636539e-03, -1.95702680e-03,  8.55938955e-04,
        -2.47340404e-04],
       [-7.13042937e-04, -4.25765415e-04, -1.00724905e-03,
        -8.22471472e-06, -2.10560623e-03,  4.56596912e-02,
    